# Homework 9
## ECE 204 Data Science & Engineering

In [1]:
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.ar_model import AutoReg
import warnings
warnings.filterwarnings("ignore")

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

---
**Problem 1.** Read in `australia-rosslyn-bay.csv`.

**What is the mean air temperature for all measurements made between 2 pm and 6 pm (Both inclusive)?**

*Note:* in 24-hour time (which is what this dataset uses, 2pm is hour 14 and 6pm is hour 18).

In [2]:
df = pd.read_csv("australia_rosslyn_bay.csv", parse_dates=True, index_col=0)
df.head()

,sea_level,water_temp,air_temp,pressure,wind_dir,wind_gust,wind_speed
datetime,,,,,,,
1997-03-26 02:00:00,1.179638,24.4,24.3,1014.8,270.0,17.9,0.0
1997-03-26 03:00:00,0.387638,24.5,24.0,1014.3,359.0,0.0,0.0
1997-03-26 04:00:00,-0.485362,24.7,22.4,1013.7,193.0,5.2,0.3
1997-03-26 06:00:00,-1.478362,24.7,22.7,1013.8,162.0,11.0,3.4
1997-03-26 07:00:00,-1.330362,24.7,22.0,1014.4,173.0,10.2,4.0


In [3]:
two_six_df = df.between_time('14:00', '18:00', inclusive='both')
mean_temp =two_six_df['air_temp'].mean()
round(mean_temp, 3)

21.594

---
**Problem 2.** Read in `australia-rosslyn-bay.csv`. **What is the mean air temperature for all measurements made in October?**

In [4]:
df = pd.read_csv("australia_rosslyn_bay.csv", parse_dates=True, index_col=0)

oct_df = df[df.index.month == 10]
oct_temp = oct_df['air_temp'].mean()
round(oct_temp, 3)

23.137

---
**Problem 3.** Read in `australia-rosslyn-bay.csv`. **What month has the lowest average pressure?** Report the answer as a number (e.g., "3" for "March").

In [5]:
df = pd.read_csv("australia_rosslyn_bay.csv", parse_dates=True, index_col=0)

pressure_df = df.groupby(df.index.month)['pressure'].mean()
lowest_avg_month = pressure_df.idxmin()
lowest_avg_month

1

---
**Problem 4.** Read in australia-rosslyn-bay.csv. **What is the 95th percentile (or quantile) of the air pressure during August?**

*Hint:* https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.quantile.html

In [6]:
df = pd.read_csv("australia_rosslyn_bay.csv", parse_dates=True, index_col=0)

august_df = df[df.index.month == 8]
nintyfifth_df = august_df['pressure'].quantile(0.95)
round(nintyfifth_df, 2)

1024.0

---

The following questions are based on a synthetic dataset "share-prices.csv" that capture daily mean share prices ((opening + closing)/ 2) of a company from 1st January, 2000 to 2nd May, 2000. This amount to 123 mean share price observations.

The next two questions will pretend you're a stock market analyst on May 2nd, 2000 and are developing a new method of finding future stock prices. You're using all days up to and including April 19th, 2000 as train data, and trying to predict stock prices on April 20th, 2000 and beyond.

Let's pretend we're developing a new technique to predict future performance. This means that there is a natural division between the train and test sets:

- **Train set:** January 1st, 2000 (2000-01-01) to 19th April, 2000 (2000-04-19), or 110 days.
- **Test set:** April 20th, 2000 (2000-04-20) to the end, 2nd May, 2000 (2000-05-02), or 13 days.

The next two questions use the library `statsmodels`, and only for the "auto-regressive" model or AR model. Auto-regression is linear regression on the last `maxlag` values of a time series (and below the statsmodel package is only used to help with predicting more than one new value).

`statsmodels` differs from Scikit-Learn in a couple ways:

* In statsmodels, the data is passed to during model creation (not during model fitting like Scikit-Learn).
* In statsmodels, the hyper-parameters are passed to during model fitting (not during model creation like Scikit-Learn).

Notably, predicting new values only needs the train dataset: the prediction works recursively starting from the last value in the train dataset.

---
**Problem 5.** Read in `share-prices.csv`. Train an auto-regressive model with the previous 4 weeks of data, or `lags=28`; that is, predict today's stock price given the last 28 days.

Use the historical data (or train set) to fit the model. Predict stock price from April, 20th 2000 to May 2nd, 2000. This will happen recursively using statsmodels – the statsmodel auto-regressive model will generate a prediction for April 20th, then use that value to predict April 21st and so on.

**What is the mean squared error between the predicted and true values from April 20th to May 2nd 2000?**

*Gold star question:* Would you use this model for actually making decisions on stock transactions? It'd be useful to plot and visualize both the predicted and true values for the testing time-period. 

In [7]:
df = pd.read_csv("share_prices.csv", parse_dates=True, index_col=0)
df.index.freq = "D"  # set as "daily" frequency; observations are recorded every day. Removes a warning in statsmodels

# Training and testing set
df_train = df.loc[:'2000-04-19']
df_test = df.loc['2000-04-20':]
df_train.tail()

,price
date,
2000-04-15,57.24
2000-04-16,42.70
2000-04-17,51.28
2000-04-18,58.82
2000-04-19,64.79


In [8]:
len(df_train), len(df_test)

(110, 13)

In [9]:
model = AutoReg(df_train.price, lags=28)
model_fit = model.fit()

start=len(df_train)
end = start + len(df_test)-1

predict = model_fit.predict(start=start, end=end)
mse_predict = mean_squared_error(df_test.price, predict)
round(mse_predict, 2)

1189.41

In [10]:
plt.figure(figsize=(10, 5))
plt.plot(df_test.index, df_test['price'], label='Actual', color='blue')
plt.plot(df_test.index, predict, label='Predicted', color='red')
plt.legend()
plt.tight_layout()
plt.show()

---
**Problem 6.** In this question, **use auto-regression model to find the best value of `lags` from 1 to 10 both inclusive**. Here **best** is defined to be **"minimum mean squared error on the validation set"**. The dataset has been split as follows:

- Training set: January 1st, 2000 to April 1st, 2000
- Validation set: from April 2nd, 2000 to May 2nd, 2000.

The validation set has 31 observations, which provides a more fair assessment of different values of `maxlag`; there are at least 20 predictions that are solely created by the model and don't use any "free" historical data.

*Gold star question: Plot the predictions for the best `lags`. Would you use this model to buy stocks?*

In [11]:
df = pd.read_csv("share_prices.csv", parse_dates=True, index_col=0)
df.index.freq = "D"  # set as "daily" frequency; observations are recorded every day. Removes a warning in statsmodels

# Training and validation set
df_train = df.loc[:'2000-04-01']
df_val = df.loc['2000-04-02':]

In [12]:
min_rmse = np.Infinity
best_k = None
best_pred = None

for k in range(1, 11):
    model = AutoReg(df_train.price, lags=k)
    model_fit = model.fit()
    
    start = len(df_train)
    end = len(df_train) + len(df_val) -1
    pred = model_fit.predict(start, end)
    rmse = mean_squared_error(df_val.price, pred)

    if rmse < min_rmse:
        min_rmse = rmse
        best_k = k
        best_pred = pred

round(min_rmse, 2), best_k

(806.51, 6)

In [13]:
plt.figure(figsize=(10, 5))
plt.plot(df_val.index, df_val['price'], label='Actual Price', marker='o')
plt.plot(df_val.index, best_pred, label=f'Predicted (lags={best_k})', marker='_')
plt.tight_layout()
plt.legend()
plt.show()